# Elastic Model of RA - FEI
Tim Tyree<br>
6.24.2020<br>

In [2]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  

#automate the boring stuff
from IPython import utils
import time, os, sys, re
beep = lambda x: os.system("echo -n '\\a';sleep 0.2;" * x)
if not 'nb_dir' in globals():
    nb_dir = os.getcwd()
sys.path.append("../lib") 
from lib import *
sys.path.append("lib") 
from lib import *

# from operari import *
# from ProgressBar import *
# from mesh_ops import *

# the visualization tools involved here for triangular meshes is
import trimesh
import pyglet
from numba import njit, cuda
# from numba.typed import List
# import numba
import trimesh

#try using a scipy sparse matrix to speed up spring force evaluations
#TODO: speed up bigger meshes with pycuda's sparce matrices
from scipy.sparse import csr_matrix
import scipy.sparse as sparse



# from pyspark import SparkContext 
# sc = SparkContext(master="local[4]")
# print(sc)


%autocall 1
%load_ext autoreload
%autoreload 2

Automatic calling is: Smart


In [3]:
import numpy.linalg as linalg

# import the watertight mesh, identifying the static caps

In [1]:
#make file for logging results
output_dir = nb_dir+'/Data/Log'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
output_fn = 'test-deform.csv'
output_file = os.path.join(output_dir, output_fn)
print(output_file)
# output_file = get_incremented_output_filename(output_dir, output_fn)
# with open(output_file, 'w') as opened_file:
#     opened_file.write('time,volume,area,avg_Fs,avg_Fp,my_Q\n')

NameError: name 'nb_dir' is not defined

In [24]:
#load mesh and reinitialize time
tme = 0.

# identify vertices that are on the caps (and therefore do not move)
# caps = trimesh.load('Data/patient85_RA_caps.ply')
mesh_closed = trimesh.load('Data/patient85_RA_closed.ply')
mesh_closed.vertices /= 10 #convert length units from mm to cm
mesh_closed.vertices -= mesh_closed.center_mass

df = pd.read_csv('Data/patient85_RA_vertices.csv',index_col = 0)
vertex_indices_of_not_caps = list(df.query('group == -1').index)
vertex_indices_of_caps = list(df.query('group > -1').index)

mesh_open = mesh_closed.submesh((vertex_indices_of_not_caps,), append=True)
mesh_caps = mesh_closed.submesh((vertex_indices_of_caps,), append=True)
mesh = mesh_closed.copy()
comp = mesh.copy()
vertices = np.array(comp.vertices)
momentum = 0.*np.array(comp.vertices)

#remove connections that are on caps
#note that this doesn't remove connections from non caps 
# to the caps, which is good.
VN = mesh.vertex_neighbors
for vid in vertex_indices_of_caps:
    VN[vid] = []

In [14]:
#assign params for isobaric expansion experiment
P = 5.*1333.    #gcm/s^2 / cm^2
k = 3000.        #gcm/s^2 / cm
extension =1.0  #unitless
h = 10**-7      #time step in seconds 
#construct constant matrices - x0 matrix and k matrix
N = len(VN)
x0_constant_mat = np.zeros((N,N)).astype('float32')
k_constant_mat = np.zeros((N,N)).astype('float32')
for i in range(N):
    for j in VN[i]:
        qi = vertices[i]
        qj = vertices[j]
        if i is not j:
            d  = np.linalg.norm(qj-qi)
            x0_constant_mat[i][j] = extension*d
            k_constant_mat[i][j] = k
        
#compute vertex masses
mass = [get_mass(vid, mesh) for vid in range(comp.vertices.shape[0])]

# define material shapes to be the initial mesh

In [16]:
# easy test: verify that the __rows__ of each triangle are the vertices and not the columns
tri = mesh.triangles[0]
assert((mesh.vertices[mesh.faces[0][0]]==tri[0]).all())

In [17]:
def precomputation():
	pass
mesh.triangles?

In [18]:
#two equivalent methods of computing triangle shape, the prior njit's better, I'd guess.
d1 = tri[1]-tri[0]
d2 = tri[2]-tri[0]



# Dm_tri = np.stack((d1,d2))
# Dm_tri = np.diff(tri,axis=0)

A_tri = np.cross(d1,d2)/2
X4 = A_tri
d3 = X4 - tri[0]
Dm_tri = np.stack((d1,d2,d3))
Dm_tri
#doesn't work
# X4 = np.mean(tri,axis=0)
# d3 = X4 - tri[0]
# Dm_tri = np.stack((d1,d2,d3))

array([[-1.10937786e-01, -1.96670055e-02,  1.80816650e-03],
       [-5.19184113e-02, -5.03691673e-02, -1.00006104e-01],
       [-5.14991111e-01, -5.60062512e-01,  2.27593660e+00]])

In [19]:

np.linalg.norm(A_tri)

0.006129214168301145

In [139]:
np.linalg.inv(Dm_tri)

array([[-10.93886775,   2.80436398,   0.13191616],
       [ 10.87595543, -16.12536163,  -0.71719929],
       [  0.20114588,  -3.33356738,   0.29274067]])

In [140]:
np.linalg.eig(Dm_tri)

(array([ 2.2992229 , -0.13393497, -0.05065828]),
 array([[ 0.00109675,  0.60610087,  0.30187889],
        [-0.04254884,  0.73637315, -0.93991942],
        [ 0.99909379,  0.30065982, -0.15943845]]))

In [141]:
np.linalg.det(Dm_tri)

0.015600033300556947

In [142]:
#rotate into the plane of tri and show d1 and d2 there
#hint
np.arccos(np.dot(d1,d2))/np.pi*180.

89.62359286324089

In [143]:
np.dot(d1,d2)

0.006569496608831287

# Extending the 3D diagonalized framework to 2D surfaces

For our familiar deformation gradient, $\mathbf{F}\in\mathbb{R}^{3\times2}$, we may decompose it as $\mathbf{F}=\mathbf{U}\hat{\mathbf{F}}\mathbf{V}^T$, where $\mathbf{U}\in\mathbb{R}^{3\times2}$ has orthonormal columns, $\hat{\mathbf{F}}\in\mathbb{R}^{2\times2}$ is diagonal, and $\mathbf{V}\in\mathbb{R}^{2\times2}$ is a rotation.

In [122]:
E,F,G = linalg.svd(Dm_tri)

In [123]:
#the triangle in the material coordinates has the same area 
# as the triangle in the initially given coordinates
A_tri = np.cross(d1,d2)/2
assert(linalg.det(np.diag(F))/2==np.linalg.norm(A_tri))

In [124]:
linalg.det(E)

-1.0000000000000004

In [125]:
np.linalg.det(G)

-1.0000000000000002

In [99]:
G

array([[ 0.82559576, -0.07634438, -0.5590735 ],
       [-0.53871076, -0.40142312, -0.74070925],
       [ 0.16787604, -0.91270533,  0.37254076]])

geez, this isn't immediately straight forward.  I'll just add damping to my ball and spring model.

# suppose there's some local elastic energy area density.


Suppose we had a local measure for elastic energy area density, $\Phi[F(X)]$, such that $E_\text{elastic}=\int_\Omega\Phi[F(X)]d^2X$ was the total (hyper)elastic potential energy for the deformed (hyperelastic) membrane.

Suppose the deformation gradient tensor field $F(X)$ takes constant values within each of our triangles, or consider $F(X)$ takes observed values on vertices and is interpolated everywhere using barycentric (linear) interpolation.

Our first guess is that our elastic energy area density takes the form $\Phi[F(X)] = \frac{k}{2}||F-\mathbb{1}||_F^2=\frac{k}{2}(F-\mathbb{1}):(F-\mathbb{1})$.

Here $F\in\mathbb{R}^{2\times2}$, (and not $F\in\mathbb{R}^{3\times3}$, ?)

---
Hmm, it seems that I could just suppose I have a typical elastic energy density distributed in a thin sheet of thickness $\delta$ and then take $\delta \rightarrow 0 $.

In [163]:
import numpy as np

def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

In [164]:
vec1 = [2, 3, 2.5]
vec2 = [-3, 1, -3.4]

mat = rotation_matrix_from_vectors(vec1, vec2)
vec1_rot = mat.dot(vec1)
assert np.allclose(vec1_rot/np.linalg.norm(vec1_rot), vec2/np.linalg.norm(vec2))

In [146]:
#todo test on d1,A_tri rotated by some rotation matrix

array([-0.11093779, -0.01966701,  0.00180817])

In [174]:
rot_example = np.array([[np.cos(np.pi/4),np.cos(np.sin(np.pi/4)),1],
 [-np.sin(np.pi/4),np.cos(np.sin(np.pi/4)),1],
 [1,1,1]])

In [183]:
mat = rotation_matrix_from_vectors(d1, rot_example.dot(d1))
mat.T.dot(d1)

array([-0.03917484, -0.06067493,  0.08649357])

In [187]:
rot_example

array([[ 0.70710678,  0.7602446 ,  1.        ],
       [-0.70710678,  0.7602446 ,  1.        ],
       [ 1.        ,  1.        ,  1.        ]])

In [180]:
rot_example.dot(d1)

array([-0.09158843,  0.06530129, -0.12879663])

# Test driven development of getting a rotation matrix 
6.25.2020<br>
Mostly works.  Tests fail between some distant triangles from mesh.triangles.  I think these would likely go away if I redid the implementation using a quaternionic algebra.


In [ ]:
# Question: can the ball-and-spring model satisfy the same tests as the corotated linear model?  
# DONE: solve the linear system of R.T.dot(a).cross(A)=0 and R.T.dot(ds1).cross(dm1)=0 for R.T symbollically in mathematica
# DONE: then, solve the linear system R.T.dot(ds1) = S.dot(dm1) & R.T.dot(ds2) = S.dot(dm2) for S symbolically in mathematica
# TODO: turn ^those into numpy functions and/or an njit'd functions

# TODO: compute, F, the deformation gradient between two example triangles
# TODO: make ^that functional and test it
# TODO: solving for $\phi$, visualize the map between the two triangles.
# TODO: compute the net force vector acting on each side of the deformed triangle
# TODO: visualize the nodal spring forces for a simple highly symmetric case
# TODO: visualize the nodal spring & pressure forces for that same case

# TODO: model the nodal forces on one vertex with a fixed 1-ring (highly-symmetric)
# TODO: time evolve ^that by FEI.  Did you use damping?
# TODO: model the nodal forces on one vertex and its 1-ring with a fixed 2-ring (highly-symmetric)
# TODO: time evolve ^that by FEI.  Did you use damping?

# TODO: can I get this to work on the full RA?
# TODO: can the ball-and-spring model satisfy the same tests?  

In [364]:
# #TODO: optimize this function by removing the first two lines (and maybe precomputing a?), and then njiting it.
# def rotation_matrix(axis, theta):
#     """
#     Return the rotation matrix associated with counterclockwise rotation about
#     the given axis by theta radians.  Uses the Euler-Rodriguez formula.
#     """
#     axis = np.asarray(axis)
#     axis = axis / np.sqrt(np.dot(axis, axis))
#     a = np.cos(theta / 2.0)
#     b, c, d = -axis * np.sin(theta / 2.0)
#     aa, bb, cc, dd = a * a, b * b, c * c, d * d
#     bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
#     return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
#                      [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
#                      [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

In [482]:
# def get_R(trim,tris,testing=True):
#     '''trim is a triangle in material/reference space that is 
#     deformed to tris, which is a triangle in real space.  
#     returns the 3x3 rotation matrix aligning both their area normals and their first shape vector.
#     get_R assumes the deformation is continuous and did not invert the triangle being deformed.'''
#     dm1 = trim[1]-trim[0]     #precompute in final algorithm
#     dm2 = trim[2]-trim[0]     #precompute in final algorithm
#     Am =  np.cross(dm1,dm2)/2 #precompute in final algorithm

#     ds1 = tris[1]-tris[0]
#     ds2 = tris[2]-tris[0]
#     As =  np.cross(ds1,ds2)/2
    
#     #Ra is a rotation_matrix that rotation_matrix rotates Ashat onto Amhat
#     Ashat = As/np.linalg.norm(As)
#     Amhat = Am/np.linalg.norm(Am)
#     v1 = Ashat
#     v2 = Amhat
#     axisa = np.cross(v1,v2)
#     thetaa = -np.arcsin(np.dot(axisa,axisa/np.linalg.norm(axisa)))
#     Ra = -rotation_matrix(axisa, thetaa)
    
#     if not np.isclose(np.dot(Ra, v1),v2).all():
#         Ra *= -1
#         print('Ra was flipped likely because of the sign ambiguity of thetaa.')
#     if testing:
#         assert(np.isclose(np.dot(Ra, v1),v2).all())#v1,v2 are about to be renamed'
        
#     #Rb is a rotation_matrix that rotates the Ra*dm1 onto ds1 without unaligning the area vectors
#     v1 = np.dot(Ra,ds1/np.linalg.norm(ds1))
#     v2 = dm1/np.linalg.norm(dm1)
#     axisb  = Amhat 
#     thetab = np.arccos(np.dot(v1,v2))
# #     axisb = np.cross(v1,v2)
# #         w = np.cross(v1,v2)
# #     thetab = np.arcsin(np.linalg.norm(w))
# #     thetab = -np.arcsin(np.dot(w,w/np.linalg.norm(w)))
#     Rb = rotation_matrix(axisb, thetab).T

#     if not np.isclose(np.dot(Rb, v1),v2).all():
#         print(np.dot(Rb, v1))
#         print(v2)
#         print('Rb was flipped likely because of the sign ambiguity of thetab.')
#         Rb = Rb.T
#         print(np.dot(Rb, v1))
#         print(v2)
        
#     R = Rb.dot(Ra).T
#     if testing:
#         assert(np.isclose(np.dot(Rb, v1),v2).all())
#         # test that Rb keeps the area vectors aligned
#         print(Rb.dot(Ra.dot(Ashat)))
#         print(Amhat)
#         assert(np.isclose(Rb.dot(Ra.dot(Ashat)),Amhat).all())
#         # test that R = Rb.dot(Ra).T rotates Amhat onto Ashat
#         assert(np.isclose(Ashat,R.dot(Amhat)).all())
#         # test that R = (Rb*Ra).T rotates dm1 onto ds1
#         assert(np.isclose(R.dot(dm1/np.linalg.norm(dm1)),ds1/np.linalg.norm(ds1)).all())
#     return R

In [487]:
trim = mesh.triangles[313]
tris = mesh.triangles[100]
get_R(trim,tris,testing=True)

Ra was flipped likely because of the sign ambiguity of thetaa.


AssertionError: 

In [367]:
trim = mesh.triangles[0]
tris = mesh.triangles[100]

dm1 = trim[1]-trim[0]
dm2 = trim[2]-trim[0]
Am =  np.cross(dm1,dm2)/2

ds1 = tris[1]-tris[0]
ds2 = tris[2]-tris[0]
As =  np.cross(ds1,ds2)/2

$\hat{v}_1\times \hat{v}_2 = \hat{\omega}\sin\vartheta$

In [370]:
#Ra is a rotation_matrix that rotation_matrix rotates Ashat onto Amhat
Ashat = As/np.linalg.norm(As)
Amhat = Am/np.linalg.norm(Am)
v1 = Ashat
v2 = Amhat
axisa = np.cross(v1,v2)
thetaa = -np.arcsin(np.dot(axisa,axisa/np.linalg.norm(axisa)))
# thetaa = np.arcsin(np.linalg.norm(axisa))
Ra = -rotation_matrix(axisa, thetaa)
print(np.dot(Ra, v1))
print(v2)
assert(np.isclose(np.dot(Ra, v1),v2).all())
if not np.isclose(np.dot(Ra, v1),v2).all():
    Ra *= -1
assert(np.isclose(np.dot(Ra, v1),v2).all())

[ 0.16787604 -0.91270533  0.37254076]
[ 0.16787604 -0.91270533  0.37254076]


In [369]:
    axisa = np.cross(v1,v2)
    thetaa = -np.arcsin(np.dot(axisa,axisa/np.linalg.norm(axisa)))
    Ra = -rotation_matrix(axisa, thetaa)

In [345]:
#Rb is a rotation_matrix that rotates the Ra*dm1 onto ds1 without unaligning the area vectors
v1 = np.dot(Ra,ds1/np.linalg.norm(ds1))
v2 = dm1/np.linalg.norm(dm1)
axisb  = Amhat #precompute in final algorithm
thetab = np.arccos(np.dot(v1,v2))
Rb = rotation_matrix(axisb, thetab)

print(np.dot(Rb, v1))
print(v2)
assert(np.isclose(np.dot(Rb, v1),v2).all())
if not np.isclose(np.dot(Rb, v1),v2).all():
    Rb *= -1
assert(np.isclose(np.dot(Rb, v1),v2).all())

[-0.98452015 -0.17453533  0.01604662]
[-0.98452015 -0.17453533  0.01604662]


In [344]:
# test that Rb keeps the area vectors aligned
assert(np.isclose(Rb.dot(Ra.dot(Ashat)),Amhat).all())

In [357]:
# test that R = Rb.dot(Ra).T rotates Amhat onto Ashat
R = Rb.dot(Ra).T
print('check that Amhat rotates to Ashat.')
print(R.dot(Amhat))
print(Ashat)
assert(np.isclose(Ashat,R.dot(Amhat)).all())

does Amhat rotate to Ashat?
[ 0.35289286  0.82841014 -0.43497502]
[ 0.35289286  0.82841014 -0.43497502]


In [360]:
# test that R = (Rb*Ra).T rotates dm1 onto ds1
print('check that dm1hat rotates to ds1hat.')
print(R.dot(dm1/np.linalg.norm(dm1)))
print(ds1/np.linalg.norm(ds1))
print('dm2hat rotates to ds2hat only if the triangles have no shear deformation between them.')
print(R.dot(dm2/np.linalg.norm(dm2)))
print(ds2/np.linalg.norm(ds2))

check that dm1hat rotates to ds1hat.
[-0.60245478 -0.15452056 -0.78305276]
[-0.60245478 -0.15452056 -0.78305276]
dm2hat rotates to ds2hat only if the triangles have no shear deformation between them.
[ 0.34642403 -0.54752627 -0.76171213]
[-0.86317263  0.46765148  0.19035522]


In [ ]:
trim = mesh.triangles[0]
tris = mesh.triangles[100]


$\vec{F}_{ij}=-\frac{1}{2}\overleftrightarrow{\mathbf{P}}(\overleftrightarrow{\mathbf{F}})\times\vec{d}_{ij}$ is the nodal force due to elasticity from node $i$ to its neighboring node $j$.

# TODO: Solve for the deformation map, $\vec{\phi}(\vec{X})$, between two given triangles

Consider for our "reference/material space", a compact topological space, $\Omega\subset\mathbb{R}^3$ that is locally two dimensional.  Let $\vec{\phi}: \Omega\rightarrow\Omega'$ be a function that continuously deforms our reference space onto another topological space, $\Omega'$, which we will refer to as our "physical space".  For some point $\vec{X}_*\in\Omega$, consider three distint points in our reference space that are located within in a sufficiently small $\epsilon$ neighborhood $\vec{X}_1,\vec{X}_2,\vec{X}_3\in N_\epsilon(\vec{X}_*)\subset\Omega$.

One can easily show for a sufficiently small choice of $\epsilon$, that by Taylor expanding $\vec{\phi}$ to leading order, we may write $\vec{\phi}$ as 
$$\vec{\phi}(\vec{X}) = \mathbf{F}(\vec{X}_*)\cdot\vec{X_i} + \vec{b}(\vec{X}_*) + \mathcal{O}(\epsilon^2),$$
where $\mathcal{O}(\epsilon^2)$ is a small correction on the order of $\epsilon^2$, $\vec{b}(\vec{X}_*)$ is a constant translation, and $\mathbf{F}(\vec{X}_*)=\partial_\vec{X}\vec{\phi}(\vec{X}_*)$ is a constant deformation, which is sometimes referred to as the _deformation gradient_ at $\vec{X}_*$.  We may write 


$\mathbf{R}\cdot\mathbf{S}$
for some translation, $\vec{b}$, some rotation $\mathbf{R}$, and some deformation, $\mathbf{S}$.  Assuming our domain is homogeneous and isotropic, we may assume $\mathbf{S}\in\mathbb{R}^{3\times3}$ is a symmetric matrix, which has six degrees of freedom.  

Consider a triangle with vertices in our stationary reference topological space, $(\vec{X}_1,\vec{X}_2,\vec{X}_3)^T$ that is continuously deformed into a triangle in the physical space $(\vec{x}_1,\vec{x}_2,\vec{x}_3)^T = (\vec{\phi}(\vec{X}_1),\vec{\phi}(\vec{X}_2),\vec{\phi}(\vec{X}_3))^T$.

Computing the translation vector, $\vec{b}$ 

In [508]:
# matrices can switch slots in np.cross
# np.cross(R+np.eye(3),d1)
# -np.cross(d1,R+np.eye(3).T)

tet = np.stack((dm1,dm2,Am))

In [528]:
#half of the determinant of tet is the squared area of the triangle.
np.linalg.det(tet)/2==Am.dot(Am)

True

In [509]:
np.linalg.inv(tet)

array([[  -8.97673571,    0.46306409,   27.3894887 ],
       [   0.20827267,   -3.39622617, -148.91066011],
       [   4.55539532,   -8.52924409,   60.78116239]])

In [510]:
np.linalg.eig(tet)

(array([-0.12517074, -0.04670493,  0.0128521 ]),
 array([[ 0.80912041,  0.28788623,  0.1449182 ],
        [ 0.58732761, -0.9507806 , -0.86859071],
        [ 0.01924665, -0.1146201 ,  0.4738659 ]]))

In [511]:
np.linalg.svd(tet)

(array([[-6.36571090e-01,  7.71218029e-01,  1.73472348e-18],
        [-7.71218029e-01, -6.36571090e-01,  0.00000000e+00],
        [-8.67361738e-19,  8.67361738e-19,  1.00000000e+00]]),
 array([0.14372308, 0.085292  , 0.00612921]),
 array([[ 0.76995429,  0.35738907,  0.5286241 ],
        [-0.61562003,  0.19809604,  0.76273845],
        [ 0.16787604, -0.91270533,  0.37254076]]))

In [512]:
np.linalg.eigvals(tet)

array([-0.12517074, -0.04670493,  0.0128521 ])

In [513]:
np.linalg.eigvalsh?

In [515]:
tet.T.dot(tet)

array([[0.01500377, 0.00479115, 0.00499391],
       [0.00479115, 0.00295514, 0.00498889],
       [0.00499391, 0.00498889, 0.0100097 ]])

# TODO: Visualize the deformation gradients of each of ^those 3 F's for the given triangle.  Do any/all of them appear reasonable?

# TODO: wrap my head around $\mathbf{P}\times\vec{d} := \mathbf{P}\cdot\vec{N}|\vec{d}|$


In [582]:
#I cannot just use np.cross to define the outward projection of a 3x3 matrix
np.cross(R,ds1).shape

(3, 3)

In [586]:
# I can, however take the cross product for each entry and then concatenate them.
def matrix_dot(P,N):
    return np.dot(P,N)
#     cx = np.dot(P[0],N)
#     cy = np.dot(P[1],d)
#     cz = np.dot(P[2],d)
#     return np.array((cx,cy,cz))
                        
# def matrix_cross(P,d):
#     cx = np.linalg.norm(np.cross(P[0],d)
#     cy = np.cross(P[1],d)
#     cz = np.cross(P[2],d)
#     return np.array((cx,cy,cz))

In [585]:
matrix_cross(R,ds1)

array([[ 0.05177626,  0.1529239 , -0.07001155],
       [ 0.09531142, -0.04920434, -0.0636199 ],
       [-0.09002641,  0.06810956,  0.0558232 ]])

# Solving for deformation gradient using quaternions

In [26]:
trim = mesh.triangles[0]
tris = mesh.triangles[100]

dm1 = trim[1]-trim[0]
dm2 = trim[2]-trim[0]
Am =  np.cross(dm1,dm2)/2

ds1 = tris[1]-tris[0]
ds2 = tris[2]-tris[0]
As =  np.cross(ds1,ds2)/2

In [27]:
from scipy.spatial.transform import Rotation

In [28]:
dm1

array([-0.11093779, -0.01966701,  0.00180817])

In [29]:
#define the constant pauli matrices as a basis for real space
sigx = np.array([[0.,1.],[1.,0.]],dtype=complex)
sigy = np.array([[1.j,0.],[0.,-1.j]],dtype=complex)
sigz = np.array([[1.,0.],[0.,-1.]],dtype=complex)

qm1 = dm1[0]*sigx + dm1[1]*sigy + dm1[2]*sigz
qm2 = dm2[0]*sigx + dm2[1]*sigy + dm2[2]*sigz
qs1 = ds1[0]*sigx + ds1[1]*sigy + ds1[2]*sigz
qs2 = ds2[0]*sigx + ds2[1]*sigy + ds2[2]*sigz

qAm =  Am[0]*sigx + Am[1]*sigy + Am[2]*sigz
qAs =  As[0]*sigx + As[1]*sigy + As[2]*sigz

In [30]:
# solve for the deformation gradient between either pair of edges.
F1 = qs1.dot(np.linalg.inv(qm1))
F2 = qs2.dot(np.linalg.inv(qm2))
FA = qAs.dot(np.linalg.inv(qAm))

In [46]:
np.trace(np.dot(F1,0.))
if np.isclose(np.trace(np.dot(F1,qs1)),0.):
    0.j

0j

In [31]:
np.linalg.eig(F1)[0]

array([1.00875401-1.07283546j, 0.83640257+1.53176966j])

In [32]:
np.linalg.eig(F2)[0]

array([ 0.1424292 +0.3924776j , -0.23089546-1.81432989j])

In [33]:
np.linalg.eig(FA)[0]

array([-0.96789163-0.41941191j, -2.8105855 +0.3131857j ])

In [34]:
np.trace(F1)

(1.8451565889452672+0.4589341919676968j)

In [35]:
np.trace(F2)

(-0.08846625814391584-1.4218522890266527j)

In [36]:
np.trace(FA)

(-3.778477131610509-0.10622620655929604j)

$(a+b)\cdot\vec{\sigma}_a\vec{\sigma}_b = (a\cdot b)\mathbb{1}+(a\times b)\vec{\sigma}_a\times\vec{\sigma}_b$

# [distraction] decompose the Piola stress tensor in terms of quaternions for some constitutive relation
try $\Psi[\mathbf{F}] = \Psi[\mathbf{S}] = \Psi(\mathbf{\Sigma}) = \frac{k}{2}||\mathbf{\Sigma}-1||^2$ implies $\Sigma_P = \vec{\chi} = q_u \oplus q_v = q_u \otimes q_v$, for linearly independent, unentangled spinors, $q_u, q_v\in \mathcal{H}$.

In [1]:
# import scikit-cuda as skcuda
# skcuda.cublasZgemmBatched(handle, transa, transb, m, n, k,
#                        alpha, A, lda, B, ldb, beta, C, ldc, batchCount):

# 

# TODO: test based development of the newmark beta method

In [5]:
# Newmark.Newmark_force?
#^this has a good outline to use, but the calculation of the input is poorly documented.